In [4]:
#!/usr/bin/env python3
import json, os
from collections import defaultdict

# Rutas a tus JSON
ARCHIVO_RELAYS = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/input_data.json'
ARCHIVO_PARAMS = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/raw/predictions_trasnformer.json'
SALIDA_MERGED = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/merged_relays.json'

def main():
    # Carga de JSON
    with open(ARCHIVO_RELAYS, 'r') as f:
        relays = json.load(f)
    with open(ARCHIVO_PARAMS, 'r') as f:
        params = json.load(f)

    # Agrupo relays y params por escenario
    relays_by_scenario = defaultdict(list)
    for entry in relays:
        relays_by_scenario[entry['scenario_id']].append(entry)

    params_by_scenario = defaultdict(list)
    for p in params:
        params_by_scenario[p['scenario_id']].append(p)

    # Combino dentro de cada escenario, ordenando params por pair_index
    for sid, relay_list in relays_by_scenario.items():
        param_list = sorted(params_by_scenario.get(sid, []), key=lambda x: x['pair_index'])
        if len(relay_list) != len(param_list):
            print(f'⚠️ Discrepancia en {sid}: {len(relay_list)} relés vs {len(param_list)} parámetros')
        for entry, p in zip(relay_list, param_list):
            entry['main_relay'].update({
                'pick_up': p['main_relay']['pick_up'],
                'TDS':      p['main_relay']['TDS']
            })
            entry['backup_relay'].update({
                'pick_up': p['backup_relay']['pick_up'],
                'TDS':      p['backup_relay']['TDS']
            })

    # Aplano la lista manteniendo orden original
    merged = list(relays)

    # Guardar resultado
    os.makedirs(os.path.dirname(SALIDA_MERGED), exist_ok=True)
    with open(SALIDA_MERGED, 'w') as f:
        json.dump(merged, f, indent=4, ensure_ascii=False)

    print(f'Se han combinado {len(merged)} registros y guardado en:\n  {SALIDA_MERGED}')

if __name__ == '__main__':
    main()


Se han combinado 6800 registros y guardado en:
  /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/merged_relays.json


In [9]:
#!/usr/bin/env python3
import json
import copy
import os
import random

# --- Constantes para el cálculo de Time_out ---
K = 0.14
N = 0.02
DECIMAL_PLACES = 4
MIN_TDS = 0.05

# --- Rutas al JSON combinado y al de salida ---
data_coordination_file = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/merged_relays.json'
output_file             = '/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_transformer.json'

# --- Función para eliminar timestamps recursivamente ---
def eliminar_timestamp(obj):
    if isinstance(obj, dict):
        obj.pop("timestamp", None)
        for valor in obj.values():
            eliminar_timestamp(valor)
    elif isinstance(obj, list):
        for elemento in obj:
            eliminar_timestamp(elemento)

# --- Función para calcular el tiempo de operación (new_timeout) ---
def calculate_operation_time(I_shc, I_pi, TDS):
    # Asegura TDS siempre positivo y al menos MIN_TDS
    try:
        TDS = abs(float(TDS))
    except (TypeError, ValueError):
        return 0.0
    if TDS < MIN_TDS:
        TDS = MIN_TDS

    # Validación de tipos y valores
    if not all(isinstance(x, (int, float)) for x in [I_shc, I_pi, TDS]):
        return 0.0
    if any(x is None or x <= 0 for x in [I_pi, TDS]) or I_shc <= 0:
        return 0.0

    # Cálculo según IEC
    try:
        M = I_shc / I_pi
    except ZeroDivisionError:
        return 0.0
    denom = (M**N) - 1
    if denom <= 0:
        return 0.0

    T = (K / denom) * TDS
    return round(T, DECIMAL_PLACES)

# --- Carga de datos ---
with open(data_coordination_file, 'r') as f:
    data = json.load(f)

# --- Procesamiento y cálculo de new_timeout ---
result = []
for entry in data:
    # Ajustes de Time_out según condiciones especiales
    mr = entry.get('main_relay', {})
    br = entry.get('backup_relay', {})

    # Caso R1 en main_relay
    if mr.get('relay') == 'R1' and mr.get('Time_out', 0) == 0:
        if br.get('Time_out', 0) != 0:
            mr['Time_out'] = br['Time_out']
        else:
            mr['Time_out'] = random.uniform(1, 3)
    # Caso R1 en backup_relay
    if br.get('relay') == 'R1' and br.get('Time_out', 0) == 0:
        if mr.get('Time_out', 0) != 0:
            br['Time_out'] = mr['Time_out']
        else:
            br['Time_out'] = random.uniform(1, 3)

    # Otros casos main_relay
    if mr.get('Time_out', 0) == 0 or mr.get('Ishc', 0) == 0:
        mr['Time_out'] = random.uniform(1, 3)

    # Otros casos backup_relay
    if br.get('Time_out', 0) == 0 or br.get('Ishc', 0) == 0:
        br['Time_out'] = random.uniform(1, 3)

    # Clonamos el entry para no modificar timestamps originales
    e = copy.deepcopy(entry)
    eliminar_timestamp(e)

    # Cálculo de new_timeout usando pick_up y TDS ajustados
    # Aseguramos que nunca sea 0: si calculate_operation_time da 0, usamos Time_out actual
    new_mr = calculate_operation_time(
        e['main_relay'].get('Ishc'),
        e['main_relay'].get('pick_up'),
        e['main_relay'].get('TDS')
    )
    if not new_mr or new_mr <= 0:
        # fallback: usar Time_out existente o aleatorio si es inválido
        new_mr = e['main_relay'].get('Time_out') or random.uniform(1, 3)
    e['main_relay']['new_timeout'] = round(new_mr, DECIMAL_PLACES)

    new_br = calculate_operation_time(
        e['backup_relay'].get('Ishc'),
        e['backup_relay'].get('pick_up'),
        e['backup_relay'].get('TDS')
    )
    if not new_br or new_br <= 0:
        new_br = e['backup_relay'].get('Time_out') or random.uniform(1, 3)
    e['backup_relay']['new_timeout'] = round(new_br, DECIMAL_PLACES)

    result.append(e)

# --- Guardar resultado ---
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w') as f:
    json.dump(result, f, indent=4, ensure_ascii=False)

print(f'Se han procesado {len(result)} entradas y guardado en "{output_file}".')


Se han procesado 6800 entradas y guardado en "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_transformer.json".
